In [90]:
import pandas as pd
import statsmodels.api as sm
import sklearn
import numpy as np

seed = 1
np.random.seed(seed)

In [91]:
df = pd.read_csv('/data/Auto.csv')
df = df.replace('?', np.nan) # assume ? is na value
df = df.dropna()

In [92]:
x = df['horsepower'].astype('int')
y = df['mpg']
x = sm.add_constant(x)
y = np.expand_dims(y, 1)
reg = sm.OLS(y, x).fit()

In [93]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.5, random_state=seed) # split data

In [95]:
from sklearn.preprocessing import PolynomialFeatures

# use polynomial features and compare MSE

def to_poly(dataset, degree=1):
    xp = np.asarray(dataset['horsepower'].astype('int')).reshape((-1, 1))
    polynomial_features = PolynomialFeatures(degree=degree)
    xp = polynomial_features.fit_transform(xp)
    return xp

def train_poly(dataset, degree=1):
    xp = to_poly(dataset, degree)
    y = dataset['mpg']
    return sm.OLS(y, xp).fit()

def test_dataset(model, dataset, degree=1):
    x = to_poly(dataset, degree)
    y = dataset['mpg']
    predict = model.predict(x)
    residuals = y - predict
    residuals = residuals * residuals
    return residuals.mean()

t1 = train_poly(train, 1)
t2 = train_poly(train, 2)
t3 = train_poly(train, 3)

print('degree 1:', test_dataset(t1, test, 1))
print('degree 2:', test_dataset(t2, test, 2))
print('degree 3:', test_dataset(t3, test, 3))

# not a significant difference between degree 2 and 3, we can look at other measures of complexity too

degree 1: 24.80212062059356
degree 2: 18.848292603275098
degree 3: 18.80511135843041


In [96]:
# cheesing with general sklearn cross validation, thanks stackoverflow
# https://stackoverflow.com/questions/41045752/using-statsmodel-estimations-with-scikit-learn-cross-validation-is-it-possible 

from sklearn.base import BaseEstimator, RegressorMixin

class SMWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, model_class, fit_intercept=True):
        self.model_class = model_class
        self.fit_intercept = fit_intercept
    def fit(self, X, y):
        if self.fit_intercept:
            X = sm.add_constant(X)
        self.model_ = self.model_class(y, X)
        self.results_ = self.model_.fit()
        return self
    def predict(self, X):
        if self.fit_intercept:
            X = sm.add_constant(X)
        return self.results_.predict(X)

In [109]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(SMWrapper(sm.OLS), x, y, cv=10, scoring='r2') # cross validation exmample
avg_error = scores.mean()
print('average error 10 fold cv for degree 1 poly:', avg_error)

average error 10 fold cv for degree 1 poly: 0.19549935010445627
